In [ ]:
import os 
os.chdir("../")

In [ ]:
%pwd


In [ ]:
from langchain_community.document_loaders import PyPDFLoader, DirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [ ]:
#extract data from the book
def load_pdf_file(data):
    loader=DirectoryLoader(data,glob="*.pdf",loader_cls=PyPDFLoader)
    documents=loader.load()
    return documents

In [5]:
extracted_data=load_pdf_file(data="Data/")

In [6]:
# extracted_data

In [7]:
# chunking og data 
def text_split(extracted_data):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=20)
    text_chunks=text_splitter.split_documents(extracted_data)
    return text_chunks

In [8]:
text_chunks=text_split(extracted_data)
print("len of text chunks",len(text_chunks))

len of text chunks 12746


In [9]:
from langchain_community.embeddings import HuggingFaceEmbeddings

In [10]:
def download_hugging_face_embeddings():
    embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings



In [11]:
embeddings=download_hugging_face_embeddings()

C:\Users\kapil\AppData\Local\Temp\ipykernel_24176\2945076228.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')


In [12]:
query_result=embeddings.embed_query("hello akg")
print("len",len(query_result))

len 384


In [13]:
from dotenv import load_dotenv
load_dotenv()


True

In [14]:
PINECONE_API_KEY=os.environ.get("PINECONE_API_KEY")
OPENAI_API_KEY=os.environ.get("OPENAI_API_KEY")

In [28]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
import os

if PINECONE_API_KEY is None:
    raise ValueError("PINECONE_API_KEY is not set. Please set the environment variable before running this cell.")

pc = Pinecone(api_key=PINECONE_API_KEY)
index_name = "medibot"

pc.create_index(
    name=index_name,
    dimension=384,
    metric="cosine",
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    )
)

{
    "name": "medibot",
    "metric": "cosine",
    "host": "medibot-oho2wkf.svc.aped-4627-b74a.pinecone.io",
    "spec": {
        "serverless": {
            "region": "us-east-1",
            "cloud": "aws",
            "read_capacity": {
                "mode": "OnDemand",
                "status": {
                    "state": "Ready",
                    "current_shards": null,
                    "current_replicas": null
                }
            }
        }
    },
    "status": {
        "ready": true,
        "state": "Ready"
    },
    "vector_type": "dense",
    "dimension": 384,
    "deletion_protection": "disabled",
    "tags": null,
    "_response_info": {
        "raw_headers": {
            "content-type": "application/json",
            "access-control-allow-origin": "*",
            "vary": "origin,access-control-request-method,access-control-request-headers",
            "access-control-expose-headers": "*",
            "x-pinecone-api-version": "2025-10",
    

In [16]:
import os 
os.environ["PINECONE_API_KEY"]=PINECONE_API_KEY
os.environ["OPENAI_API_KEY"]=OPENAI_API_KEY

In [29]:
from langchain_pinecone import Pinecone

docsearch = Pinecone.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embeddings
)

In [21]:
# load the index

from langchain_pinecone import Pinecone

docsearch=Pinecone.from_existing_index(
    index_name=index_name,
    embedding=embeddings,
)

In [22]:
retirver=docsearch.as_retriever(search_type="similarity",search_kwargs={"k":3})

In [23]:
retirved_docs=retirver.invoke("acne")
print(retirved_docs)

[Document(metadata={'creationdate': '2004-12-28T15:13:17-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-28T14:23:27-06:00', 'page': 21, 'page_label': '22', 'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'source': 'Data\\Medical_book.pdf', 'total_pages': 602}, page_content='GALE ENCYCLOPEDIA OF ALTERNATIVE MEDICINE 2 5\nAcne\nAcne vulgaris affecting a woman’s face. Acne is the general\nname given to a skin disorder in which the sebaceous\nglands become inflamed. (Photograph by Biophoto Associ-\nates, Photo Researchers, Inc. Reproduced by permission.)\nAcne\nDefinition\nAcne is a common inflammatory skin disease char-\nacterized by pimples on the face, chest, and back. It oc-\ncurs when the pores of the skin become clogged with oil,\ndead skin cells, and/or bacteria.'), Document(metadata={'creationdate': '2004-12-28T15:13:17-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-28T14:23:27-06:00', 'page': 21, 'page_label': '22', 'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'source': 'Data\\Medical_book.p

In [24]:
from langchain_openai import OpenAI
llm=OpenAI(temperature=0.4,max_tokens=500)

In [25]:
from langchain_classic.chains.retrieval import create_retrieval_chain
from langchain_classic.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt=(
    "you are an assistant for question-answering tasks."
    "Use the following pieces of retrieved context to answer"
    "the question. If you dont know just say that you dont know.Use Three sentences maximum and keep the anwswers concise."
    "\n\n"
    "{context}"
)

prompt=ChatPromptTemplate.from_messages(
    [
        ("system",system_prompt),
        ("human","{input}")
    ]
)

In [26]:
question_answer_chain=create_stuff_documents_chain(llm,prompt)
rag_chain=create_retrieval_chain(retirver,question_answer_chain)

In [27]:
response=rag_chain.invoke({"input":"what is acne"})
print(response["answer"])

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}